In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Nov  3 15:08 kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c Kannada-MNIST

  0% 0.00/33.1k [00:00<?, ?B/s]
100% 33.1k/33.1k [00:00<00:00, 32.5MB/s]
  0% 0.00/818k [00:00<?, ?B/s]
100% 818k/818k [00:00<00:00, 54.2MB/s]
  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 125MB/s]
 52% 5.00M/9.71M [00:00<00:00, 50.7MB/s]
100% 9.71M/9.71M [00:00<00:00, 61.9MB/s]


In [6]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import pandas as pd

In [0]:
data=pd.read_csv('/content/train.csv')

In [7]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
!unzip test.csv.zip

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [0]:
data = pd.read_csv('/content/train.csv')
datat = pd.read_csv('/content/test.csv')

In [65]:
img_train = data.drop(["label"], axis=1).values.reshape(-1, 28, 28, 1).astype('float32')
img_label = data["label"]
img_test = datat.drop(["id"], axis=1).values.reshape(-1, 28, 28, 1).astype('float32')
print("img_train.shape = ", img_train.shape)
print("img_label.shape = ", img_label.shape)
print("img_test.shape = ", img_test.shape)


img_train.shape =  (60000, 28, 28, 1)
img_label.shape =  (60000,)
img_test.shape =  (5000, 28, 28, 1)


In [0]:
from sklearn.model_selection import train_test_split
X_data = img_train / 255
Y_data = to_categorical(img_label)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [97]:
x_test.shape,y_test.shape

((12000, 28, 28, 1), (12000, 10))

In [0]:
import ktrain
from ktrain import vision 

In [0]:
data_aug = vision.get_data_aug(featurewise_center=False, 
                               featurewise_std_normalization=False,
                               horizontal_flip=True,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.0,
                               rotation_range=10)

In [0]:
(train_data, val_data, preproc)=vision.images_from_array(x_train, y_train, 
                                                  validation_data=(x_test, y_test),
                                                  data_aug=data_aug)


In [92]:
model = vision.image_classifier('default_cnn', train_data, val_data)

Is Multi-Label? False
default_cnn model created.


In [0]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, 
                             )

In [94]:
learner.fit_onecycle(1e-3, 1)




begin training using onecycle policy with max lr of 0.001...
Epoch 1/1
1500/1500 [==============================] - 216s 144ms/step - loss: 0.5284 - acc: 0.8270 - val_loss: 0.0575 - val_acc: 0.9825


In [95]:
loss, acc = learner.model.evaluate_generator(val_data, steps=len(val_data))
print('final loss:%s, final score:%s' % (loss, acc))



final loss:0.05754931269586086, final score:0.9825


In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [101]:
learner.model.save('my_model.h5')

ValueError: ignored

In [88]:
pred=learner.predict(preproc)

TypeError: ignored

In [0]:
predction=pd.DataFrame(pred)
sub_df=pd.read_csv('/content/sample_submission.csv')
dataset=pd.concat([sub_df['id'],predction],axis=1)


In [87]:
dataset

,id,0,1,2,3,4,5,6,7,8,9
0,0.0,9.999981e-01,1.040824e-07,1.121480e-06,2.832548e-10,9.389331e-12,1.135050e-11,1.766009e-11,1.396493e-10,7.260599e-07,1.169830e-11
1,1.0,6.473990e-05,5.462038e-11,1.379424e-08,2.120507e-11,2.738583e-06,8.147806e-08,1.691375e-09,3.129805e-09,9.999324e-01,4.681346e-08
2,2.0,2.394400e-07,1.929696e-07,5.806961e-08,1.049815e-05,6.807159e-06,9.998412e-01,1.338179e-09,7.664705e-08,1.407574e-04,2.099262e-08
3,3.0,6.564072e-10,3.761617e-09,1.925149e-09,3.932832e-07,9.999976e-01,2.034320e-06,1.283887e-08,7.913871e-10,4.507167e-10,1.104111e-08
4,4.0,1.556811e-10,1.487210e-09,8.783658e-10,7.142398e-10,1.101732e-06,2.147881e-10,1.263693e-07,2.664319e-08,1.897112e-06,9.999968e-01
...,...,...,...,...,...,...,...,...,...,...,...
11995,NaN,1.834846e-05,2.534244e-06,9.998180e-01,5.903394e-07,5.982564e-09,3.027462e-07,2.477064e-06,1.570608e-04,5.598338e-07,2.410002e-09
11996,NaN,8.118548e-11,2.367957e-10,2.506245e-12,5.948060e-10,9.999872e-01,3.773870e-07,8.138619e-09,2.070908e-09,5.438277e-07,1.190119e-05
11997,NaN,7.645837e-10,1.110646e-08,1.254451e-08,2.010435e-07,5.972599e-03,9.940269e-01,2.664609e-10,7.246213e-10,2.495505e-07,6.613270e-11
11998,NaN,1.338786e-11,2.911795e-10,1.467078e-10,9.393996e-10,5.556620e-12,1.601310e-13,9.999882e-01,4.814168e-08,6.000938e-11,1.183243e-05


In [86]:
dataset.columns=['id','labels']
dataset.to_csv('sample.csv',index=False)

ValueError: ignored